# User must run cell below

In [1]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")
export_matrices = mm.tool("tmg.input_output.export_binary_matrix")  


def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"

# User Input

In [5]:
pth = r"C:\Users\PECHEN\Desktop\407 - Megazone Adjustment Matrices" #Folder with matrices you want to reindex
matrix_list = ["AM_mzagg_factors","PM_mzagg_factors"]
# matrix_list = ["am_phf"]
# matrix_list = ["pm_phf"]
output_pth = r"C:\Users\PECHEN\Desktop\407 - Megazone Adjustment Matrices\2041" # Output folder to save matrices, if you pick same folder old files will be overwritten

sc_or = 2016 #Scenario with old zone system
sc_new = 2041 #Scenario with new zone system
mat_id = "mf16" #Dummy matrix to import demand (must exist)
pad = "1" #Padding factor to overwrite 0s. For PHF matrix, use 0.469 for AM and 0.307 for PM

# Cell below reindexes

In [6]:
for name in matrix_list:
    import_matrices(4,int(mat_id[-2:]),os.path.join(pth, name + ".mtx"),sc_or,name)
    mcSpec = {
        "type": "MATRIX_CALCULATION",
        "result": mat_id,
        "expression":  pad,
        "aggregation": {
            "origins": None,
            "destinations": None
        },
        "constraint": {
            "by_value": {
            "interval_min": 0,
            "interval_max": 0,
            "condition": "INCLUDE",
            "od_values": mat_id
        },
            "by_zone": None
        }
    }
    matrix_calculator(mcSpec,scenario = eb.scenario(sc_new))
    export_matrices(4,mat_id[-2:],os.path.join(output_pth,name + ".mtx"),sc_new)